In [1]:
# pour les imports
import numpy as np

# Zayed

In [97]:
class Cards : 
    def __init__(self):
        self.ncards = 20 # number of cards
        self.cards = 4*[7,8,9,10,11] # cards (?? 11 pour "A")
    
    def shuffle(self):
        '''Knuth shuffle'''
        n=self.ncards
        for i in range(1,n):
            j = np.random.randint(0,i+1) # random int betwwen [0,i]
            a = self.cards[i]
            b = self.cards[j]
            self.cards[i] = b
            self.cards[j] = a
    def hands(self):
        '''distribution of cards by the agent :)'''
        self.hand1 = [self.cards[0], self.cards[2]]
        self.hand2 = [self.cards[1], self.cards[3]]
        self.sumHand1 = self.cards[0]+self.cards[2]# the sum of the two cards
        self.sumHand2 = self.cards[1]+self.cards[3] # for agent (agent is the one who shuffles)
        
    


In [111]:
#test
cards=Cards()
cards.shuffle()
cards.hands()
print(cards.hand1)
print(cards.hand2)

[9, 7]
[10, 8]


# Abdallahi

[7, 8, 9, 10, 11, 7, 8, 9, 10, 11, 7, 8, 9, 10, 11, 7, 8, 9, 10, 11]
[11, 10, 7, 8, 7, 9, 11, 9, 11, 10, 11, 9, 8, 9, 7, 7, 10, 8, 8, 10]


# Mermoud